# Walrun, Pythonic Redis Client

## 简介

[walrus](https://github.com/coleifer/walrus): 基于 ``redis-py``, 实现了一些基于 redis 的 Python 数据结构. 你可以将其理解为提供了基于 redis 的分布式 dict, list, set 实现.

- Redis Hash = Python dict
- Redis List = Python list
- Redis Set = Python set
- Redis Sorted (ZSet) = Python ordered by value dict
- Redis HyperLogLog: 用于估算大约有多少个不同的元素的数据结构.
- Redis Array:
- Redis BitField (BitMap, 位图):
- Redis BloomFilter (布隆过滤器): 用于快速判断一个元素是否在集合中. 如果说不在, 那一定不在. 如果说在, 则有极小概率不在. 并且布隆过滤器只能添加不能删除元素.

In [1]:
# 连接到 AWS ElasticCache Redis Cluster
import redis
import walrus

endpoint = "sanhe-dev.hozbo8.ng.0001.use1.cache.amazonaws.com"

r = redis.Redis(host=endpoint, port=6379, db=0)
db = walrus.Walrus(host=endpoint, port=6379, db=0)

## Redis Hash | Python dict

In [2]:
# 初始化 Hash 对象
r.delete("my_dict")
dct = db.Hash("my_dict")
dct

<Hash "my_dict": {}>

In [3]:
# 基本的 set, get 操作
dct["name"] = "alice"
print(f'dct["name"] = {dct["name"]}')
print(f'dct.get("name") = {dct.get("name")}')
print(f'dct.get("invalid_name") = {dct.get("invalid_name")}')
print(f'dct.get("invalid_name", "default-value") = {dct.get("invalid_name", "default-value")}')

# 判断 key 是否在字典中
print(f'"name" in dct = {"name" in dct}')
print(f'"invalid_name" in dct = {"invalid_name" in dct}')

dct["name"] = b'alice'
dct.get("name") = b'alice'
dct.get("invalid_name") = None
dct.get("invalid_name", "default-value") = default-value
"name" in dct = True
"invalid_name" in dct = False


In [4]:
# update() 操作
dct.update(name="bob", email="bob@email.com")
print(f'dct = {dct}')

# atomic increment
dct.update(value=0)
dct.incr("value", 1)
print(f'dct["value"] = {dct["value"]}')

dct.incr("value", 2)
print(f'dct["value"] = {dct["value"]}')

dct = <Hash "my_dict": {b'name': b'bob', b'email': b'bob@email.com'}>
dct["value"] = b'1'
dct["value"] = b'3'


In [5]:
# keys(), values(), items() 元素遍历操作
print(f"list(dct.keys()) = {list(dct.keys())}")
print(f"list(dct.values()) = {list(dct.values())}")
print(f"list(dct.items()) = {list(dct.items())}")

list(dct.keys()) = [b'name', b'email', b'value']
list(dct.values()) = [b'bob', b'bob@email.com', b'3']
list(dct.items()) = [(b'name', b'bob'), (b'email', b'bob@email.com'), (b'value', b'3')]


In [6]:
# 将 Hash 转化成 dict
dct.as_dict(decode=True)

{'name': 'bob', 'email': 'bob@email.com', 'value': '3'}

In [7]:
# 将 dict 转化成 Hash
dct1 = walrus.Hash.from_dict(db, "my_dict1", dct.as_dict())
dct1

<Hash "my_dict1": {b'name': b'bob', b'email': b'bob@email.com', b'value': b'3'}>

In [8]:
# clear() 操作
dct.clear()
dct

<Hash "my_dict": {}>

## Redis List | Python list

In [9]:
r.delete("my_list")
# 初始化 List 对象, 开始是空列表
lst = db.List("my_list")
lst

<List "my_list": >

In [10]:
# 添加一些元素
lst.extend([1, 2, 3])
lst


<List "my_list": 1, 2, 3>

In [11]:
# Redis List 中的元素只能是字符串, 整数添加进去后会变成字符串
lst[0] == 1

False

In [12]:
lst[0] == "1".encode("ascii")

True

In [13]:
# 下面的方法是从 Python 对象构造
r.delete("my_list")
lst = walrus.List.from_list(db, "my_list", range(3))
lst

<List "my_list": 0, 1, 2>

In [14]:
r.delete("my_list")
lst = walrus.List.from_list(db, "my_list", ["a", "b", "c"])
lst

<List "my_list": a, b, c>

In [15]:
r.delete("my_list")
lst = walrus.List.from_list(db, "my_list", "xyz")
lst

<List "my_list": x, y, z>

In [16]:
r.delete("my_list")
lst = db.List("my_list")
lst.extend(["a", "b", "c"])
lst.as_list(decode=True)

['a', 'b', 'c']

In [17]:
r.delete("deque")
dq = db.List("deque")
dq.extend(["m", "n"])
dq.append("o")
dq.prepend("l")
dq

<List "deque": l, m, n, o>

In [18]:
dq.pop()

b'o'

In [19]:
dq

<List "deque": l, m, n>

In [20]:
dq.popleft()

b'l'

In [21]:
dq

<List "deque": m, n>

Slice 操作

In [22]:
lst = walrus.List.from_list(db, "slice_lst", "abcdefg")
lst

<List "slice_lst": a, b, c, d, e, f, a, b, c, d...>

In [23]:
lst[0]

b'a'

In [24]:
lst[2]

b'c'

In [25]:
lst[-1]

b'g'

In [26]:
lst[:2]

[b'a', b'b']

In [27]:
lst[2:]

[b'c', b'd', b'e', b'f', b'a', b'b', b'c', b'd', b'e', b'f', b'g']

In [28]:
lst[:-2]

[b'a', b'b', b'c', b'd', b'e', b'f', b'a', b'b', b'c', b'd', b'e']

In [29]:
lst[-2:]

[b'f', b'g']

## Redis Set | Python set

In [30]:
r.delete("set1")
s1 = walrus.Set.from_set(db, "set1", "xy")
s1

<Set "set1": 2 items>

In [31]:
r.delete("set2")
s2 = walrus.Set.from_set(db, "set2", "yz")
s2

<Set "set2": 2 items>

In [32]:
s1.members()

{b'x', b'y'}

In [33]:
for i in s1:
    print(i)

b'y'
b'x'


In [34]:
s2.members()


{b'y', b'z'}

In [35]:
for i in s2:
    print(i)

b'z'
b'y'


In [36]:
s1 | s2

{b'x', b'y', b'z'}

In [37]:
s1 & s2

{b'y'}

In [38]:
s1 - s2

{b'x'}

In [39]:
s2 - s1

{b'z'}

## Redis Sorted (ZSet) | Python ordered-set

In [40]:
r.delete("zset1")
z = walrus.ZSet.from_dict(db, "zset1", dict(x=5, y=3, z=7))
z

<ZSet "zset1": y, x, z>

In [41]:
for i in z.iterator(reverse=True):
    print(i)


b'z'
b'x'
b'y'
